In [1]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
import transformers
from transformers import BertTokenizer
from IPython.display import clear_output

In [3]:
PRETRAINED_MODEL_NAME = "ckiplab/bert-base-chinese"  # 指定繁簡中文 BERT-BASE 預訓練模型

from transformers import (
   BertTokenizerFast,
   AutoModelForMaskedLM,
   AutoModelForCausalLM,
   AutoModelForTokenClassification,
   AutoModel,
)


tokenizer = BertTokenizerFast.from_pretrained(PRETRAINED_MODEL_NAME)


In [4]:
vocab = tokenizer.vocab
print("字典大小：", len(vocab))

字典大小： 21128


In [5]:
import pandas as pd

In [6]:
#from google.colab import drive
#drive.mount('/content/drive')

In [7]:
#df_train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/data/movie/basic4_data.csv")
df_train=pd.read_csv("animes8_data_replace2.csv")
#df_train=pd.read_csv("basic8_data_augment.csv") #testing
df_val=pd.read_csv("animes8_val.csv")
df_total=pd.read_csv("animes8_total.csv")
#df_val_replace=pd.read_csv("basic8_val_replace.csv")
len_=len(df_train)
animes=["鬼滅之刃","間諜家家酒","五等分的新娘","出租女友","進擊的巨人","咒術迴戰","無職轉生","Re從零開始的異世界生活"]
print("data len=",len_)
print("val len=",len(df_val))

data len= 4000
val len= 1202


In [8]:
#-------把太長的去掉------
MAX_LENGTH=80

df_train = df_train[~(df_train.sentence.apply(lambda x : len(x)> MAX_LENGTH))]
df_val = df_val[~(df_val.sentence.apply(lambda x : len(x)> MAX_LENGTH))]
df_total = df_val[~(df_total.sentence.apply(lambda x : len(x)> MAX_LENGTH))]
#df_val_replace=df_val_replace[~(df_val_replace.sentence.apply(lambda x : len(x)> MAX_LENGTH))]
len_=len(df_train)
print("after len=",len_)
print("after len=",len(df_val))

after len= 2743
after len= 1090


C:\Users\user\AppData\Local\Temp\ipykernel_22392\3786618029.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_total = df_val[~(df_total.sentence.apply(lambda x : len(x)> MAX_LENGTH))]


In [9]:
# idempotence, 將處理結果另存成 tsv 供 PyTorch 使用

print("訓練樣本數：", len(df_train))
print(df_train.head())
print(df_val.head())

訓練樣本數： 2743
                                             sentence label
5   如題 剛剛活動解到第二彈發現不但有 某人無慘還有 竈門某人欸欸 這次活動是和鬼滅聯動嗎那後續...  鬼滅之刃
10  作畫上來看，兩個可以算不相上下吧，劇情才是兩者帶來最大的差異，鬼滅直白簡單的王道劇情，Fat...  鬼滅之刃
11                        Fate跟鬼滅比起來哪個更能表現幽浮社的實力當代表作?  鬼滅之刃
12  如題為何只有墮g的臉有血絲其他人都沒？另外為什麼叫墮姬墮過胎的關係嗎？如果墮胎改成剁胎會不會...  鬼滅之刃
13  某人那紫藤花毒，最強那種連上弦都能殺，為什麼不給其他柱分發一些有這種毒的暗器，其他柱不夠上弦...  鬼滅之刃
                                            sentence label
0        故事背景為日本大正時代，描述主角為了尋求讓被變成鬼的妹妹恢復為人類的方法，踏上斬鬼之旅  鬼滅之刃
1  平安時代一名罹患絕症的青年鬼舞辻無慘在受到一位醫師的治療後成為鬼，鬼雖具有超長壽命和強悍的身...  鬼滅之刃
2                            無慘希望找到能夠克服陽光的方法，便製造許多手下  鬼滅之刃
3                旗下實力強大的十二隻鬼會被授予十二鬼月的稱號，強大的鬼可使用異能血鬼術  鬼滅之刃
7  起初炭治郎以為鬼不會存在，直至隔天早上他返家後發現他的家人慘遭殺害身亡多時，而大妹禰豆子雖然...  鬼滅之刃


In [10]:
df_train.label.value_counts() / len(df_train)
df_train.to_csv("train.tsv", sep="\t", index=False)
df_train.label.value_counts() / len(df_train)

進擊的巨人           0.153846
Re從零開始的異世界生活    0.136712
咒術迴戰            0.130879
無職轉生            0.123952
五等分的新娘          0.121400
間諜家家酒           0.117025
出租女友            0.111192
鬼滅之刃            0.104995
Name: label, dtype: float64

In [11]:
df_val.to_csv("val.tsv", sep="\t", index=False)
df_val.label.value_counts() / len(df_val)

Re從零開始的異世界生活    0.222018
進擊的巨人           0.171560
出租女友            0.125688
無職轉生            0.111927
鬼滅之刃            0.094495
五等分的新娘          0.094495
咒術迴戰            0.090826
間諜家家酒           0.088991
Name: label, dtype: float64

In [12]:
#if data is unbalance https://stackoverflow.com/questions/57244781/deleting-large-amount-of-data-from-pandas-dataframe
"""
bad_label_index=df_val[df_val["label"]=="血觀音"].sample(60).index
df_val = df_val[~df_val.index.isin(bad_label_index)]
t=df_val.index.isin(bad_label_index)
"""
"""
percentage=df_val.label.value_counts() / len(df_val)
while max(percentage)>0.20:
    for serie in series:
        if percentage[serie]>0.20:
            bad_label_index=df_val[df_val["label"]==serie].sample(len(df_val)//20).index
            df_val = df_val[~df_val.index.isin(bad_label_index)]
    percentage=df_val.label.value_counts() / len(df_val)
print(percentage)


df_val.to_csv("val.tsv", sep="\t", index=False)
"""

'\npercentage=df_val.label.value_counts() / len(df_val)\nwhile max(percentage)>0.20:\n    for serie in series:\n        if percentage[serie]>0.20:\n            bad_label_index=df_val[df_val["label"]==serie].sample(len(df_val)//20).index\n            df_val = df_val[~df_val.index.isin(bad_label_index)]\n    percentage=df_val.label.value_counts() / len(df_val)\nprint(percentage)\n\n\ndf_val.to_csv("val.tsv", sep="\t", index=False)\n'

In [13]:
#if data is unbalance https://stackoverflow.com/questions/57244781/deleting-large-amount-of-data-from-pandas-dataframe
"""
bad_label_index=df_val[df_val["label"]=="血觀音"].sample(60).index
df_val = df_val[~df_val.index.isin(bad_label_index)]
t=df_val.index.isin(bad_label_index)
"""
"""
percentage=df_total.label.value_counts() / len(df_total)
while max(percentage)>0.20:
    for serie in series:
        if percentage[serie]>0.20:
            bad_label_index=df_total[df_total["label"]==serie].sample(len(df_total)//20).index
            df_total = df_total[~df_total.index.isin(bad_label_index)]
    percentage=df_total.label.value_counts() / len(df_total)
print(percentage)
"""
df_total.to_csv("total.tsv", sep="\t", index=False)

tokens_tensor：代表識別每個 token 的索引值，用 tokenizer 轉換即可  

segments_tensor：用來識別句子界限。第一句所有index對應 0，第二句所有index對應 1。另外注意句子間的 [SEP] 為 0  

masks_tensor：用來界定自注意力機制範圍。1 讓 BERT 關注該位置，0 則代表是 padding 不需關注  //因為我們會zero padding讓batch裡大小一致


In [14]:
label_map = {anime:i  for i, anime in enumerate(animes)}
inv_label_map = {v: k for k, v in label_map.items()}
print(inv_label_map)


def indices_to_labels(indices):
    for i in range(len(indices)):
        indices[i] = inv_label_map[indices[i]]
    return indices


print(indices_to_labels([2, 3, 4]))

{0: '鬼滅之刃', 1: '間諜家家酒', 2: '五等分的新娘', 3: '出租女友', 4: '進擊的巨人', 5: '咒術迴戰', 6: '無職轉生', 7: 'Re從零開始的異世界生活'}
['五等分的新娘', '出租女友', '進擊的巨人']


In [15]:
"""
實作一個可以用來讀取訓練 / 測試集的 Dataset，這是你需要徹底了解的部分。
此 Dataset 每次將 tsv 裡的一筆成對句子轉換成 BERT 相容的格式，並回傳 3 個 tensors：
- tokens_tensor：兩個句子合併後的索引序列，包含 [CLS] 與 [SEP]
- segments_tensor：可以用來識別兩個句子界限的 binary tensor
- label_tensor：將分類標籤轉換成類別索引的 tensor, 如果是測試集則回傳 None
"""
from torch.utils.data import Dataset

#https://pytorch.org/tutorials/beginner/data_loading_tutorial.html
class SeriesDataset(Dataset):
    # 讀取前處理後的 tsv 檔並初始化一些參數
    def __init__(self, mode, tokenizer,label_map):
        #assert mode in ["train", "test","val"]  # 確定檔案名稱只接受train或test
        self.mode = mode
        # 大數據你會需要用 iterator=True
        self.df = pd.read_csv(mode + ".tsv", sep="\t").fillna("")   #讀處裡好的tsv避免跟所有檔案的csv搞混
        self.len = len(self.df)
        self.label_map = label_map
        self.tokenizer = tokenizer  # 我們將使用 BERT tokenizer

    # 定義回傳一筆訓練 / 測試數據的函式
    def __getitem__(self, idx):
        """
        if self.mode == "test":
            text, label = self.df.iloc[idx, :].values
            label_tensor = None
        else:
        """
        text,label = self.df.iloc[idx, :].values
        # 將 label 文字也轉換成索引方便轉換成 tensor
        label_id = self.label_map[label]
        label_tensor = torch.tensor(label_id)

        # 建立第一個句子的 BERT tokens 並加入分隔符號 [SEP]
        word_pieces = ["[CLS]"]
        tokens = self.tokenizer.tokenize(text)
        word_pieces += tokens + ["[SEP]"]
        
        # 將整個 token 序列轉換成索引序列
        ids = self.tokenizer.convert_tokens_to_ids(word_pieces)
        tokens_tensor = torch.tensor(ids)

        return (tokens_tensor, label_tensor)

    def __len__(self):
        return self.len

In [16]:
# 初始化一個專門讀取訓練樣本的 Dataset，使用中文 BERT 斷詞
trainset = SeriesDataset("train", tokenizer=tokenizer,label_map=label_map)     #tokenizer 在上面注意力圖 上有宣告
valset=SeriesDataset("val", tokenizer=tokenizer,label_map=label_map)
totalset=SeriesDataset("total", tokenizer=tokenizer,label_map=label_map)
#valset_replace = SeriesDataset("val_replace", tokenizer=tokenizer,label_map=label_map)     #tokenizer 在上面注意力圖 上有宣告
print(trainset.df)
print(valset.df)
#print(valset_replace.df)

                                               sentence         label
0     如題 剛剛活動解到第二彈發現不但有 某人無慘還有 竈門某人欸欸 這次活動是和鬼滅聯動嗎那後續...          鬼滅之刃
1     作畫上來看，兩個可以算不相上下吧，劇情才是兩者帶來最大的差異，鬼滅直白簡單的王道劇情，Fat...          鬼滅之刃
2                           Fate跟鬼滅比起來哪個更能表現幽浮社的實力當代表作?          鬼滅之刃
3     如題為何只有墮g的臉有血絲其他人都沒？另外為什麼叫墮姬墮過胎的關係嗎？如果墮胎改成剁胎會不會...          鬼滅之刃
4     某人那紫藤花毒，最強那種連上弦都能殺，為什麼不給其他柱分發一些有這種毒的暗器，其他柱不夠上弦...          鬼滅之刃
...                                                 ...           ...
2738                      即便心念是想過者平靜的生活，或心靈比教會更來得無害善良也是  Re從零開始的異世界生活
2739                              然而民間有流傳，說是半獸人的出現是因為魔女  Re從零開始的異世界生活
2740    因此在這先入為主的觀念下，導致男主半獸人傭兵對魔術與魔女本身就沒什好感，甚至可以說是厭惡與畏懼  Re從零開始的異世界生活
2741                                    何況是建構在魔術體系上的魔法呢  Re從零開始的異世界生活
2742                                 但他卻與零，創造魔法的魔女搭上了關係  Re從零開始的異世界生活

[2743 rows x 2 columns]
                                               sentence         label
0           故事背景為日本大正時代，描述主角為了尋求讓被變成鬼的妹妹恢復為人類的方法，踏上斬鬼之旅          

In [17]:
import random
# 選擇一個樣本
sample_idx = int(random.random()*len(trainset))
print(sample_idx)
# 將原始文本拿出做比較
text,label = trainset.df.iloc[sample_idx]

# 利用剛剛建立的 Dataset 取出轉換後的 id tensors
tokens_tensor,label_tensor = trainset[sample_idx]

# 將 tokens_tensor 還原成文本
tokens = tokenizer.convert_ids_to_tokens(tokens_tensor.tolist())
combined_text = "".join(tokens)     #tokens 是一個很多字元的list用.join把他轉成string

# 渲染前後差異，毫無反應就是個 print。可以直接看輸出結果
print(f"""[原始文本]
句子 1：{text}
分類  ：{label}

--------------------

[Dataset 回傳的 tensors]
tokens_tensor  ：{tokens_tensor}

label_tensor   ：{label_tensor}
--------------------

[還原 tokens_tensors]
{combined_text}
""")

942
[原始文本]
句子 1：頂級的畫工，一流的人設
分類  ：出租女友

--------------------

[Dataset 回傳的 tensors]
tokens_tensor  ：tensor([ 101, 7515, 5159, 4638, 4529, 2339, 8024,  671, 3837, 4638,  782, 6257,
         102])

label_tensor   ：3
--------------------

[還原 tokens_tensors]
[CLS]頂級的畫工，一流的人設[SEP]



In [18]:
print(type(trainset[0]))
print(len(trainset))
print(type(trainset))

#trainset 直接用[]來看是一個(2657,)個tuple of three tensor

<class 'tuple'>
2743
<class '__main__.SeriesDataset'>


In [19]:
"""
實作可以一次回傳一個 mini-batch 的 DataLoader
這個 DataLoader 吃我們上面定義的 `MovieDataset`，
回傳訓練 BERT 時會需要的 4 個 tensors：
- tokens_tensors  : (batch_size, max_seq_len_in_batch)
- segments_tensors: (batch_size, max_seq_len_in_batch)
- masks_tensors   : (batch_size, max_seq_len_in_batch)
- label_ids       : (batch_size)
"""

from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

# 這個函式的輸入 `samples` 是一個 list由前面的3個tenors組成，裡頭的每個 element 都是
# 剛剛定義的 `MovieDataset` 回傳的一個樣本，每個樣本都包含 3 tensors：
# - tokens_tensor
# - segments_tensor
# - label_tensor
# 它會對前兩個 tensors 作 zero padding，並產生前面說明過的 masks_tensors
def create_mini_batch(samples):
    tokens_tensors = [s[0] for s in samples]       #把每個sample的tensor取出來放進list 產生一個 list of tensor (batch_size,)
    
    # 測試集有 labels
    if samples[0][1] is not None:
        label_ids = torch.stack([s[1] for s in samples])
    else:
        label_ids = None
    
    # zero pad 到同一序列長度(要同時做好幾個RNN就要batch中每個元素一樣長)
    #https://blog.csdn.net/qq_43391414/article/details/123289492
    
    tokens_tensors = pad_sequence(tokens_tensors, 
                                  batch_first=True)
    
    return tokens_tensors, label_ids


In [20]:
#把data跟寫好的create_mini_batch放進trainloader
BATCH_SIZE = 32
trainloader = DataLoader(trainset, batch_size=BATCH_SIZE,
                         shuffle=True,
                         collate_fn=create_mini_batch
                         )

valloader = DataLoader(valset, batch_size=8,
                         shuffle=True,
                         collate_fn=create_mini_batch
                         )
totalloader = DataLoader(valset, batch_size=8,
                         shuffle=True,
                         collate_fn=create_mini_batch
                         )
"""
valloader_replace = DataLoader(valset_replace, batch_size=BATCH_SIZE,
                         shuffle=True,
                         collate_fn=create_mini_batch
                         )
"""

'\nvalloader_replace = DataLoader(valset_replace, batch_size=BATCH_SIZE,\n                         shuffle=True,\n                         collate_fn=create_mini_batch\n                         )\n'

In [21]:
data = next(iter(trainloader))  # 取出traing loader 裡的一個mini_batch

tokens_tensors, label_ids = data

print(f"""
tokens_tensors.shape   = {tokens_tensors.shape} 
{tokens_tensors}
------------------------
label_ids.shape        = {label_ids.shape}
{label_ids}
""")


tokens_tensors.shape   = torch.Size([32, 66]) 
tensor([[ 101, 3312, 3469,  ...,    0,    0,    0],
        [ 101, 6857, 4415,  ...,    0,    0,    0],
        [ 101, 2769, 2205,  ...,    0,    0,    0],
        ...,
        [ 101, 3378,  782,  ...,    0,    0,    0],
        [ 101, 1333, 1993,  ...,    0,    0,    0],
        [ 101, 5445,  844,  ...,    0,    0,    0]])
------------------------
label_ids.shape        = torch.Size([32])
tensor([1, 7, 6, 1, 1, 2, 2, 2, 2, 5, 7, 0, 7, 1, 2, 1, 7, 2, 0, 5, 5, 6, 6, 5,
        2, 0, 0, 3, 3, 5, 4, 4])



建立 BERT 用的 mini-batch 時最需要注意的就是 zero padding 的存在了。  
你可以發現除了 lable_ids 以外，其他 3 個 tensors 的每個樣本的最後大都為 0，  
這是因為每個樣本的 tokens 序列基本上長度都會不同，需要補 padding。

In [22]:
from transformers import BertForSequenceClassification

In [23]:
import os
PRETRAINED_MODEL_NAME = "ckiplab/bert-base-chinese"
NUM_LABELS = len(label_map)
PATH="model_anime_sam100.pt"

model = BertForSequenceClassification.from_pretrained(
        PRETRAINED_MODEL_NAME, num_labels=NUM_LABELS,attention_probs_dropout_prob=0.10,
        hidden_dropout_prob=0.10)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
#uncommend to load model

if os.path.isfile(PATH):
    print("=====>LOAD checkpoint"+PATH)
    checkpoint = torch.load(PATH)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
else:
    print("======>LOAD new model")


Some weights of the model checkpoint at ckiplab/bert-base-chinese were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ckiplab/bert-base-c

======>LOAD new model


In [24]:
print(model.bert.embeddings.word_embeddings.weight.requires_grad)
model.bert.embeddings.word_embeddings.weight.requires_grad=False
print(model.bert.embeddings.word_embeddings.weight.requires_grad)

True
False


In [25]:
print("""
name            module
----------------------""")
for name, module in model.named_children():
    if name == "bert":
        for n, _ in module.named_children():
            print(f"{name}:{n}")
    else:
        print("{:15} {}".format(name, module))


name            module
----------------------
bert:embeddings
bert:encoder
bert:pooler
dropout         Dropout(p=0.1, inplace=False)
classifier      Linear(in_features=768, out_features=8, bias=True)


In [26]:
""""
定義一個可以針對特定 DataLoader 取得模型預測結果以及分類準確度的函式
之後也可以用來生成上傳到 Kaggle 競賽的預測結果

2019/11/22 更新：在將 `tokens`、`segments_tensors` 等 tensors
丟入模型時，強力建議指定每個 tensor 對應的參數名稱，以避免 HuggingFace
更新 repo 程式碼並改變參數順序時影響到我們的結果。
"""

def get_predictions(model, dataloader, compute_acc=False):
    predictions = None
    correct = 0
    total = 0
      
    with torch.no_grad():
        # 遍巡整個資料集
        wrong_list=[]
        for data in dataloader:
            # 將所有 tensors 移到 GPU 上
            if next(model.parameters()).is_cuda:     #檢查model是否在GPU
                data = [t.to("cuda:0") for t in data if t is not None]    #sentence 跟label tensor從tuple改放到cuda list
            tokens_tensors= data[0]   #data 已經放在cuda了 如果沒有的話 應該會報錯
            outputs = model(input_ids=tokens_tensors)
            #ouputs 原本是 SequenceClassifierOutput class
            #用outputs[0]可以取出他的tensor部分 shape=(64,3)
            logits = outputs[0]
            _, pred = torch.max(logits.data, 1)   #後面的如果是0回傳整column中最大的,1的話回傳整row最大的
 
            # 用來計算訓練集的分類準確率
            if compute_acc:
                labels = data[1]
                total += labels.size(0)
                correct += (pred == labels).sum().item()   #.item把tensor 轉為純量
                
                cur=0
                for i in (pred!=labels):
                    if i==True:
                        wrong_list.append((tokens_tensors[cur],pred[cur].item(),labels[cur].item()))
                    cur+=1
                
            # 將當前 batch 記錄下來
            if predictions is None:
                predictions = pred
            else:
                predictions = torch.cat((predictions, pred))  #最後會形成一長串所有的prediction
    
    if compute_acc:
        acc = correct / total
        return wrong_list, acc
    return predictions
    

In [27]:

# 讓模型跑在 GPU 上並取得訓練集的分類準確率
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device:", device)
model = model.to(device)     #把model移到devive 應該會是cuda:0
acc_record =0.5 #set 0 to start saving model
train_acc_list=[]
val_acc_list=[]

device: cuda:0


In [28]:
#for data in trainloader:
    #print(data[0].shape)
print(next(model.parameters()).shape)

torch.Size([21128, 768])


In [29]:
data = next(iter(trainloader))    #並沒有紀錄iter 所以不會一直next下去 只會取trainloader的開頭 並取得next
for t in data:
    print(t.shape)
tmp=[t.to("cuda:0") for t in data if t is not None]
print(type(tmp))
print(type(data))

torch.Size([32, 81])
torch.Size([32])
<class 'list'>
<class 'tuple'>


In [30]:
from sam import SAM

from utility.smooth_cross_entropy import smooth_crossentropy
from utility.initialize import initialize
from utility.step_lr import StepLR
from utility.bypass_bn import enable_running_stats, disable_running_stats

输入：
    input_ids：训练集，torch.LongTensor类型，shape是[batch_size, sequence_length]
    token_type_ids：可选项，当训练集是两句话时才有的。
    attention_mask：可选项，当使用mask才有，可参考原论文。
    labels：数据标签，torch.LongTensor类型，shape是[batch_size]

In [31]:
%%time

# 訓練模式
model.train()

# 使用 Adam Optim 更新整個分類模型的參數
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5) #1e-4 train不起來


EPOCHS = 15
FREQ=10
acc_record=0.5

for epoch in range(EPOCHS):

    running_loss = 0.0
    running_correct=0
    running_count=0
    model.train()
    
    for i,data in enumerate(trainloader):

        tokens_tensors, labels = [t.to(device) for t in data]

        # 將參數梯度歸零
        optimizer.zero_grad()

        # forward pass
        outputs = model(input_ids=tokens_tensors,
                        labels=labels)

        loss = outputs[0]
        logits=outputs[1]
        
        _, pred = torch.max(logits.data, 1)   #後面的如果是0回傳整column中最大的,1的話回傳整row最大的
        correct = (pred == labels).sum().item()   #.item把tensor 轉為純量
        if i%FREQ==0:            
            print("epoch={} iteration={} acc={}".format(epoch,i,correct/labels.size(0)))
        # backward
        loss.backward()
        optimizer.step()

        # 紀錄當前 batch loss
        running_loss += loss.item()
        running_count+= labels.size(0)
        running_correct+=correct
    # 計算分類準確率
    model.eval()
    acc=running_correct/running_count
    with torch.no_grad():
        _, val_acc = get_predictions(model, valloader, compute_acc=True)
    train_acc_list.append(acc)
    val_acc_list.append(val_acc)
    print('[epoch %d] loss: %.3f, acc: %.3f,val acc: %.3f'%
          (epoch + 1, running_loss, acc, val_acc))
    PATH = "model_anime_origin.pt"
    if val_acc > acc_record:
        acc_record = val_acc
        torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
        }, PATH)
        print("Save Checkpoint to:", PATH)


epoch=0 iteration=0 acc=0.0
epoch=0 iteration=10 acc=0.09375
epoch=0 iteration=20 acc=0.0625
epoch=0 iteration=30 acc=0.21875
epoch=0 iteration=40 acc=0.3125
epoch=0 iteration=50 acc=0.3125
epoch=0 iteration=60 acc=0.21875
epoch=0 iteration=70 acc=0.21875
epoch=0 iteration=80 acc=0.28125
[epoch 1] loss: 164.244, acc: 0.281,val acc: 0.314
epoch=1 iteration=0 acc=0.3125
epoch=1 iteration=10 acc=0.5625
epoch=1 iteration=20 acc=0.375
epoch=1 iteration=30 acc=0.59375
epoch=1 iteration=40 acc=0.5
epoch=1 iteration=50 acc=0.65625
epoch=1 iteration=60 acc=0.5625
epoch=1 iteration=70 acc=0.40625
epoch=1 iteration=80 acc=0.4375
[epoch 2] loss: 119.521, acc: 0.532,val acc: 0.460
epoch=2 iteration=0 acc=0.625
epoch=2 iteration=10 acc=0.8125
epoch=2 iteration=20 acc=0.625
epoch=2 iteration=30 acc=0.46875
epoch=2 iteration=40 acc=0.8125
epoch=2 iteration=50 acc=0.65625
epoch=2 iteration=60 acc=0.71875
epoch=2 iteration=70 acc=0.71875
epoch=2 iteration=80 acc=0.625
[epoch 3] loss: 85.791, acc: 0.688,

In [32]:
%%time
#SAM to train
# 訓練模式
model.train()


#optimizer = torch.optim.Adam(model.parameters(), lr=1e-5) #1e-4 train不起來
base_optimizer = torch.optim.SGD
optimizer = SAM(model.parameters(), base_optimizer, rho=2.0, adaptive=True, lr=0.00003, momentum=0.9)
#optimizer = SAM(model.parameters(), base_optimizer, rho=2.0, adaptive=True, lr=0.0003, momentum=0.9)
EPOCHS = 5
FREQ=10
#acc_record=0.5

for epoch in range(EPOCHS):

    running_loss = 0.0
    running_correct=0
    running_count=0
    model.train()
    
    for i,data in enumerate(trainloader):

        tokens_tensors, labels = [t.to(device) for t in data]

        # 將參數梯度歸零
        optimizer.zero_grad()
        
        # first forward-backward step
        enable_running_stats(model)
        outputs = model(input_ids=tokens_tensors,
                labels=labels)
        logits=outputs[1]
        
        loss = smooth_crossentropy(logits, labels)
        loss.mean().backward()
        optimizer.first_step(zero_grad=True)

        # second forward-backward step
        disable_running_stats(model)     
        outputs = model(input_ids=tokens_tensors,
                labels=labels)
        logits=outputs[1]
        smooth_crossentropy(logits, labels).mean().backward()
        optimizer.second_step(zero_grad=True)
        
        
        _, pred = torch.max(logits.data, 1)   #後面的如果是0回傳整column中最大的,1的話回傳整row最大的
        correct = (pred == labels).sum().item()   #.item把tensor 轉為純量
        if i%FREQ==0:            
            print("epoch={} iteration={} acc={}".format(epoch,i,correct/labels.size(0)))

        # 紀錄當前 batch loss
        running_loss += loss.mean().item()
        running_count+= labels.size(0)
        running_correct+=correct
    # 計算分類準確率
    model.eval()
    acc=running_correct/running_count
    with torch.no_grad():
        _, val_acc = get_predictions(model, valloader, compute_acc=True)
        train_acc_list.append(acc)
        val_acc_list.append(val_acc)
        print('[epoch %d] loss: %.3f, acc: %.3f,val acc: %.3f'%
              (epoch + 1, running_loss, acc, val_acc))
        if val_acc > acc_record:
            acc_record = val_acc
            SAVE_PATH = "model_anime_sam.pt"
            torch.save({
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
            }, SAVE_PATH)
            print("Save Checkpoint to:", SAVE_PATH)


epoch=0 iteration=0 acc=0.96875
epoch=0 iteration=10 acc=0.84375
epoch=0 iteration=20 acc=0.875
epoch=0 iteration=30 acc=0.90625
epoch=0 iteration=40 acc=0.875
epoch=0 iteration=50 acc=0.96875
epoch=0 iteration=60 acc=0.71875
epoch=0 iteration=70 acc=0.9375
epoch=0 iteration=80 acc=0.65625
[epoch 1] loss: 16.761, acc: 0.821,val acc: 0.559
epoch=1 iteration=0 acc=0.84375
epoch=1 iteration=10 acc=1.0
epoch=1 iteration=20 acc=0.9375
epoch=1 iteration=30 acc=0.84375
epoch=1 iteration=40 acc=0.9375
epoch=1 iteration=50 acc=0.96875
epoch=1 iteration=60 acc=0.9375
epoch=1 iteration=70 acc=0.9375
epoch=1 iteration=80 acc=0.875
[epoch 2] loss: 15.660, acc: 0.903,val acc: 0.551
epoch=2 iteration=0 acc=0.96875
epoch=2 iteration=10 acc=0.84375
epoch=2 iteration=20 acc=0.90625
epoch=2 iteration=30 acc=0.96875
epoch=2 iteration=40 acc=1.0
epoch=2 iteration=50 acc=0.9375
epoch=2 iteration=60 acc=0.4375
epoch=2 iteration=70 acc=0.84375
epoch=2 iteration=80 acc=0.90625
[epoch 3] loss: 14.815, acc: 0.90

In [33]:
PATH="model_anime_sam.pt"
if os.path.isfile(PATH):
    print("=====>LOAD checkpoint"+PATH)
    checkpoint = torch.load(PATH)
    model.load_state_dict(checkpoint['model_state_dict'])
    model = model.to(device)
    print("to",device)

In [42]:
%%time
#SAM to Full data
# 訓練模式
model.train()

# 使用 Adam Optim 更新整個分類模型的參數
#optimizer = torch.optim.Adam(model.parameters(), lr=1e-5) #1e-4 train不起來
base_optimizer = torch.optim.SGD
optimizer = SAM(model.parameters(), base_optimizer, rho=2.0, adaptive=True, lr=0.00003, momentum=0.9)
#optimizer = SAM(model.parameters(), base_optimizer, rho=2.0, adaptive=True, lr=0.001, momentum=0.9)
EPOCHS = 12
FREQ=10
#acc_record=0.5

for epoch in range(EPOCHS):

    running_loss = 0.0
    running_correct=0
    running_count=0
    model.train()
    
    for i,data in enumerate(totalloader):

        tokens_tensors, labels = [t.to(device) for t in data]

        # 將參數梯度歸零
        optimizer.zero_grad()
        
        # first forward-backward step
        enable_running_stats(model)
        outputs = model(input_ids=tokens_tensors,
                labels=labels)
        logits=outputs[1]
        
        loss = smooth_crossentropy(logits, labels)
        loss.mean().backward()
        optimizer.first_step(zero_grad=True)

        # second forward-backward step
        disable_running_stats(model)     
        outputs = model(input_ids=tokens_tensors,
                labels=labels)
        logits=outputs[1]
        smooth_crossentropy(logits, labels).mean().backward()
        optimizer.second_step(zero_grad=True)
        
        
        _, pred = torch.max(logits.data, 1)   #後面的如果是0回傳整column中最大的,1的話回傳整row最大的
        correct = (pred == labels).sum().item()   #.item把tensor 轉為純量
        if i%FREQ==0:            
            print("epoch={} iteration={} acc={}".format(epoch,i,correct/labels.size(0)))

        # 紀錄當前 batch loss
        running_loss += loss.mean().item()
        running_count+= labels.size(0)
        running_correct+=correct
    # 計算分類準確率
    model.eval()
    acc=running_correct/running_count
    with torch.no_grad():
        _, val_acc = get_predictions(model, valloader, compute_acc=True)
    train_acc_list.append(acc)
    val_acc_list.append(val_acc)
    print('[epoch %d] loss: %.3f, acc: %.3f,val acc: %.3f'%
          (epoch + 1, running_loss, acc, val_acc))
    if val_acc > acc_record:
        acc_record = val_acc
        SAVE_PATH = "model_anime_sam"+str(int(val_acc*100))+".pt"
        torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
        }, SAVE_PATH)
        print("Save Checkpoint to:", SAVE_PATH)


epoch=0 iteration=0 acc=0.375
epoch=0 iteration=10 acc=0.625
epoch=0 iteration=20 acc=0.25
epoch=0 iteration=30 acc=0.75
epoch=0 iteration=40 acc=0.5
epoch=0 iteration=50 acc=0.375
epoch=0 iteration=60 acc=0.625
epoch=0 iteration=70 acc=0.5
epoch=0 iteration=80 acc=0.625
epoch=0 iteration=90 acc=0.625
epoch=0 iteration=100 acc=0.375
epoch=0 iteration=110 acc=0.625
epoch=0 iteration=120 acc=0.75
epoch=0 iteration=130 acc=0.625
[epoch 1] loss: 63.860, acc: 0.561,val acc: 0.822
Save Checkpoint to: model_anime_sam82.pt
epoch=1 iteration=0 acc=0.625
epoch=1 iteration=10 acc=0.75
epoch=1 iteration=20 acc=0.625
epoch=1 iteration=30 acc=0.5
epoch=1 iteration=40 acc=0.625
epoch=1 iteration=50 acc=0.75
epoch=1 iteration=60 acc=0.625
epoch=1 iteration=70 acc=0.625
epoch=1 iteration=80 acc=0.5
epoch=1 iteration=90 acc=0.75
epoch=1 iteration=100 acc=0.75
epoch=1 iteration=110 acc=0.625
epoch=1 iteration=120 acc=0.125
epoch=1 iteration=130 acc=0.75
[epoch 2] loss: 59.676, acc: 0.583,val acc: 0.825
S

In [35]:
#print wrong predict of val
model.eval()
wrong_list, acc = get_predictions(model, valloader, compute_acc=True)
print(acc)
print(wrong_list[0])
for wrong in wrong_list:
    tokens = tokenizer.convert_ids_to_tokens(wrong[0])
    combined_text = "".join(tokens)     #tokens 是一個很多字元的list用.join把他轉成string
    combined_text=combined_text.replace("[PAD]","")
    print(combined_text)
    print("predict:{} true:{}".format(inv_label_map[wrong[1]],inv_label_map[wrong[2]]))

0.808256880733945
(tensor([ 101, 2641,  785, 1558, 7029, 5953, 5948, 4158,  862,  889, 1168, 3346,
         776, 8024, 1961, 6303, 6245, 1339, 6965,  678, 8024, 1058, 2552, 2361,
        3307, 5543, 6878, 1168, 5156, 5251,  102,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
       device='cuda:0'), 2, 5)
[CLS]悠仁問野薔薇為何來到東京，她說討厭鄉下，內心希望能遇到紗織[SEP]
predict:五等分的新娘 true:咒術迴戰
[CLS]而這時萊納腦海中浮現出的記憶是[UNK][UNK]？[SEP]
predict:Re從零開始的異世界生活 true:進擊的巨人
[CLS]而將兩人的狀況看在眼中的一花，給兩人提出建議[SEP]
predict:Re從零開始的異世界生活 true:五等分的新娘
[CLS]待人真誠的富家千金，也是安妮亞的第一個好朋友，自進校首日以來便與安妮亞形影不離[SEP]
predict:無職轉生 true:間諜家家酒
[CLS]接下來，左右命運的舞台的幕布升起[SEP]
predict:Re從零開始的異世界生活 true:出租女友
[CLS]不知過了多久，在禮拜堂的地下醒來的艾倫發現身體被鎖鏈所拘束，眼前站著的是羅德和希斯特利亞[SEP]
predict:Re從零開始的異世界生活 true:進擊的巨人
[CLS]在第12卷中因為不慎踩到轉移迷宮的陷阱，在滿是魔物的迷宮中獨自戰鬥長達一個月，瀕臨死亡時被魯迪救出並對魯迪產生愛慕[SEP]
predict:Re從零開始的異世界生活 true:無職轉生
[CLS]由於魔力枯竭和極度恐懼頭髮變成了白色，以菲茲的身份女扮男裝擔任起愛麗兒的貼身護衛[SEP]
predict:Re從零開始的異世界生活 true:無職轉生
[CLS]千鶴醒來，發現那裡是小島的淺灘[SEP]
predict:鬼

[CLS]在第9卷中贏下魔王巴迪岡迪的挑戰，成為學校受人尊敬的存在[SEP]
predict:Re從零開始的異世界生活 true:無職轉生
[CLS]魯迪烏斯、艾莉絲及瑞傑路德因為魔物調查的委託前往石化之森，但那裡卻有兩組接到相同依賴的冒險者[SEP]
predict:Re從零開始的異世界生活 true:無職轉生
[CLS]旁人多叫他為達米安少爺，安妮亞則稱他為次子[SEP]
predict:Re從零開始的異世界生活 true:間諜家家酒
[CLS]成績很差，但也一直在努力，而成績最高的考卷科目為古語，洛伊德推測她過去曾居住在使用古語的國家[SEP]
predict:進擊的巨人 true:間諜家家酒
[CLS]能夠預知未來，安妮亞也能透過讀心術看到彭德的預知畫面，不過他自己針對預知所作的思考和行動有時比安妮亞還要單純[SEP]
predict:Re從零開始的異世界生活 true:間諜家家酒
[CLS]但是，二人偶然在同一所大學重逢，並確定是同一棟公寓的鄰居[SEP]
predict:無職轉生 true:出租女友
[CLS]公式書裡提到，鬍子最長的部分為17公分[SEP]
predict:Re從零開始的異世界生活 true:間諜家家酒
[CLS]二乃和三玖一起租用上杉母親原來的料理店，開了咖啡廳[SEP]
predict:出租女友 true:五等分的新娘
[CLS]夏油、真人跟漏瑚討論要把20跟手指收集完之後全交給宿[UNK][SEP]
predict:鬼滅之刃 true:咒術迴戰
[CLS]原來乙骨是日本三大怨靈之一菅原道真的後代，里香的詛咒是乙骨在拒絕她死亡時，無意識下的產物[SEP]
predict:鬼滅之刃 true:咒術迴戰
[CLS]在轉移事件中轉移到了阿斯拉王國的王宮中，被愛麗兒和盧克所救[SEP]
predict:Re從零開始的異世界生活 true:無職轉生
[CLS]砲彈向艾倫他們射來[SEP]
predict:Re從零開始的異世界生活 true:進擊的巨人
[CLS]為了找出咒術師究竟是誰，昴向拉姆和雷姆建議提前先去村里採買生活必需品[SEP]
predict:咒術迴戰 true:Re從零開始的異世界生活
[CLS]兒時擁抱尤利的時候因為用力過度把他弄到骨折折斷兩根肋骨，以至於之後觸碰小孩都格外小心[SEP]
predict:Re從零開始的異世界生

In [36]:
#plot
#plt.xlim(0,2200 ) # 設定 x 軸座標範圍
import matplotlib.pyplot as plt
plt.ylim(0, 1) # 設定 y 軸座標範圍

plt.xlabel('epoch', fontsize="10") # 設定 x 軸標題內容及大小
plt.ylabel('Accuracy', fontsize="10") # 設定 y 軸標題內容及大小
plt.title('Accuracy in use replace data (dropout=0.1)(8 movie prediction)', fontsize="18") # 設定圖表標題內容及大小

plt.plot(range(len(train_acc_list)),train_acc_list,marker='o', color='blue',label="Training Accuracy")
plt.plot(range(len(val_acc_list)),val_acc_list,marker='o', color='red',label="Validation Accuracy")
#for x,y in zip(range(len(val_acc_list)),val_acc_list): 
#    plt.text(x, y+0.01, "({}, {:.3f})".format(x,y))

plt.legend()
fig = plt.gcf()
fig.set_size_inches(15, 10)
plt.savefig("Movie Accuracy use replace data (dropout=0.1).png",dpi=100)

plt.clf()

<Figure size 1500x1000 with 0 Axes>

In [43]:
model.eval()
test_text = "[CLS]沒有血緣的一家人[SEP]"  # 盡量湊到6個字以上 ex:xx到底是甚麼
tokens = tokenizer.tokenize(test_text)
ids = tokenizer.convert_tokens_to_ids(tokens)

id_tensor = torch.tensor([ids]).to(device)
print(test_text)
print(tokens[:])
print(ids[:])
k = 8
with torch.no_grad():
    outputs = model(input_ids=id_tensor)
    prediction = outputs[0]
    probs, indices = torch.topk(torch.softmax(prediction, -1), k)
    probs = probs.tolist()
    indices = indices.tolist()
    probs = probs[0]
    indices = indices[0]
for i, (indice, p) in enumerate(zip(indices, probs), 1):
    label = inv_label_map[indice]
    print("Top {} ({:2}%)：{}".format(i, int(p * 100), label[:10]))

[CLS]沒有血緣的一家人[SEP]
['[CLS]', '沒', '有', '血', '緣', '的', '一', '家', '人', '[SEP]']
[101, 3760, 3300, 6117, 5225, 4638, 671, 2157, 782, 102]
Top 1 (28%)：間諜家家酒
Top 2 (15%)：進擊的巨人
Top 3 (15%)：無職轉生
Top 4 (14%)：Re從零開始的異世界
Top 5 (12%)：出租女友
Top 6 (10%)：五等分的新娘
Top 7 ( 2%)：咒術迴戰
Top 8 ( 1%)：鬼滅之刃
